# Trabalho Prático 3
**Grupo 22**

Alexis Correia - A102495 <br>
João Fonseca - A102512 <br>

## Problema 1

O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 

> Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.

```Python
{INPUT  a, b}
{assume  a > 0 and b > 0}

0: r, r', s, s', t, t' = a, b, 1, 0, 0, 1
1: while r' != 0:
2:   q = r div r'
3:   r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 

{OUTPUT r, s, t}
```

1. Construa um SFOTS usando BitVector’s de tamanho $n$ que descreva o comportamento deste programa.  Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.
2. Prove, usando a metodologia dos invariantes e interpolantes, que o modelo nunca atinge o estado de erro.

## Resolução

Um SFOTS é definido  por $\:\:\Sigma\;\equiv\;\langle\,\mathsf{X}\,,\,\mathsf{next}\,,\,\mathsf{I}\,,\,\mathsf{T}\,,\,\mathsf{E}\,\rangle\:\:$. Similiar a um FOTS, os estados são constituídos pelas variáveis do programa mais o __program counter__ (pc) e, tanto o estado inicial quanto as relações de trnsição, são caracterizados por predicados. A maior diferença se encontra na existência do estado de erro.

- As variáveis do programa são: $ r $, $ r' $, $ s $, $ s' $, $ t $, $ t' $, $ q $ e não podemos nos esquecer do $ a $ e do $ b $
- O estado inicial é caracterizado pelo predicado: $\mathit{pc} = 0 \wedge a \ge 0 \wedge b \ge 0$
- As transições possíveis são caracterizadas das seguintes formas:
$$
\begin{array}{c}
(\mathit{pc}=0\wedge a\ge 0\wedge b\ge 0\wedge\mathit{PC}=1\wedge A=a\wedge B=b\wedge R=a\wedge R'=b\wedge S=1\wedge S'=0\wedge T=0\wedge T'=1\wedge Q=q)
\\\vee\\
(\mathit{pc}=1\wedge r'=0\wedge\mathit{PC}=4\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge Q=q)
\\\vee\\
(\mathit{pc}=1\wedge r'\neq 0\wedge\mathit{PC}=2\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge Q=q)
\\\vee\\
(\mathit{pc}=2\wedge\mathit{PC}=3\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge Q=r\;\;\text{div}\;\; r')
\\\vee\\
(\mathit{pc}=3\wedge\mathit{PC}=1\wedge A=a\wedge B=b\wedge R=r'\wedge R'=r-q\times r'\wedge S=s'\wedge S'=s-q\times s'\wedge T=t'\wedge T'=t-q\times t'\wedge Q=q)
\\\vee\\
(\mathit{pc}=4\wedge\mathit{PC}=4\wedge A=a\wedge B=b\wedge R=r\wedge R'=r'\wedge S=s\wedge S'=s'\wedge T=t\wedge T'=t'\wedge Q=q)
\end{array}
$$
- O estado de erro acontece caso haja um ``overflow`` em alguma das variáveis ou quando $ r=0 $

> Repare que: nas transições, dado que as variáveis $ r' $, $ s' $ e $ t' $ são variáveis do programa, representamos as variáveis do próximo estado por letras maiúsculas.

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import BVType, INT

n = 32 # num bits = 4 bytes

# SFOTS #
def genState(vars, s, i):
    state = {}
    for v in vars:
        if v=='pc':
            state[v] = Symbol(v+'!'+s+str(i), INT)
        else:    
            state[v] = Symbol(v+'!'+s+str(i), BVType(n))
    return state

def init(state):
    A = BVSGE(state['a'], BV(0,n)) 
    B = BVSGE(state['b'], BV(0,n))
    C = Equals(state['pc'], Int(0))
    return And(A,B,C)

def trans(curr, prox):
    t01 = And(Equals(curr['pc'],Int(0)), BVSGE(curr['a'],BV(0,n)), BVSGE(curr['b'],BV(0,n)), Equals(prox['pc'],Int(1)),
              Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']), Equals(prox['r'],curr['a']),
              Equals(prox['r\''],curr['b']), Equals(prox['s'],Int(1)), Equals(prox['s\''],Int(0)),
              Equals(prox['t'],Int(0)), Equals(prox['t'], Int(1)), Equals(prox['q'],curr['q']))
    
    t14 = And(Equals(curr['pc'],Int(0)), Equals(curr['r\''], BV(0,n)), Equals(prox['pc'],Int(4)), Equals(prox['a'],curr['a']),
              Equals(prox['b'],curr['b']), Equals(prox['r'],curr['r']), Equals(prox['r\''],curr['r\'']),
              Equals(prox['s'],curr['s']), Equals(prox['s\''],curr['s\'']), Equals(prox['t'],curr['t']), 
              Equals(prox['t'], curr['t\'']), Equals(prox['q'],curr['q']))
    
    t12 = And(Equals(curr['pc'],Int(0)), Not(Equals(curr['r\''], BV(0,n))), Equals(prox['pc'],Int(2)), Equals(prox['a'],curr['a']),
              Equals(prox['b'],curr['b']), Equals(prox['r'],curr['r']), Equals(prox['r\''],curr['r\'']),
              Equals(prox['s'],curr['s']), Equals(prox['s\''],curr['s\'']), Equals(prox['t'],curr['t']), 
              Equals(prox['t'], curr['t\'']), Equals(prox['q'],curr['q']))
    
    t23 = And(Equals(curr['pc'],Int(0)), Equals(prox['pc'],Int(3)), Equals(prox['a'],curr['a']),
              Equals(prox['b'],curr['b']), Equals(prox['r'],curr['r']), Equals(prox['r\''],curr['r\'']),
              Equals(prox['s'],curr['s']), Equals(prox['s\''],curr['s\'']), Equals(prox['t'],curr['t']), 
              Equals(prox['t'], curr['t\'']), Equals(prox['q'],Div(curr['r'],curr['r\''])))

    t31 = And(Equals(curr['pc'], Int(3)), Equals(prox['pc'], Int(1)), Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']),
              Equals(prox['r'],curr['r\'']), Equals(prox['r\''], Minus(curr['r'], Times(curr['q'],curr['r\'']))),
              Equals(prox['s'],curr['s\'']), Equals(prox['s\''], Minus(curr['s'], Times(curr['q'],curr['s\'']))),
              Equals(prox['t'],curr['t\'']), Equals(prox['t\''], Minus(curr['t'], Times(curr['q'],curr['t\'']))),
              Equals(prox['q'], curr['q']))
    
    t44 = And(Equals(curr['pc'], Int(4)), Equals(prox['pc'], Int(4)), Equals(prox['a'],curr['a']), Equals(prox['b'],curr['b']),
              Equals(prox['r'],curr['r']), Equals(prox['s'],curr['s']), Equals(prox['t'],curr['t']), Equals(prox['r\''],curr['r\'']),
              Equals(prox['s\''],curr['s\'']), Equals(prox['t\''],curr['t\'']), Equals(prox['q'],curr['q']))

    return Or(t01, t14, t12, t23, t31, t44)

def error(state):
    of_R = BVSGT(state['q'], Div(BV(2**n -1, n), state['r\'']))
    of_S = BVSGT(state['q'], Div(BV(2**n -1, n), state['s\'']))
    of_T = BVSGT(state['q'], Div(BV(2**n -1, n), state['t\'']))
    overflow = Or(of_R, of_S, of_T)
    R = Equals(state['r'], BV(0, n))
    return Or(overflow, R)

def genTrace(vars,init,trans,error,n): 
    with Solver(name="z3") as s:
        X = [genState(vars,'X',i) for i in range(n+1)]
        I = init(X[0])
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]
        E = [error(X[i]) for i in range(n)]
        if s.solve([I,And(Tks),Not(And(E))]):
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v])) ###
        else:
            print("ERRO")

var = ['pc', 'a', 'b', 'r', 'r\'', 's', 's\'', 't', 't\'', 'q']

...Inverso...

In [ ]:
def invert(trans): # genTrace: <var, error, invert(trans), init, n>
    return lambda curr, nxt: trans(nxt, curr)

Com o SFOTS devidamente escrito, podemos testar se este é, de facto, seguro. Para isso,  utilizaremos a metodologia dos invariantes e interpolantes e demonstraremos que o modelo nunca atinge o estado de erro.

In [ ]:
import itertools

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

In [ ]:
def model_checking(vars,init,trans,error,N,M):
    with Solver(name="z3") as solver:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        transt = invert(trans)
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        for (n,m) in order:      
            #passo 2
            I = init(X[0])
            Tn = And(trans(X[i], X[i+1]) for i in range(n))
            Rn = And(I, Tn)

            E = error(Y[0])
            Bm = And(transt(Y[i], Y[i+1]) for i in range(m))
            Um = And(E, Bm)

            Vnm = And(Rn, same(X[n],Y[m]), Um)
            with Solver as solver([Vnm]):
                if solver.solve():
                    print(">O sistema é seguro.")
                    return
            #passo 3
            A = And(Rn, same(X[n],Y[m]))
            B = Um
            C = binary_interpolant(A,B)
            if C is None:
                print("Interpolante: ",C)
                return
            #passo 4
            C0 = rename(C, X[0])
            T = trans(X[0], X[1])
            C1 = rename(C, X[1])
            if not solver.solve(C0, T, Not(C1)):
                print("O sistema é seguro")
            else:
                #passo 5.1
                Cn = rename(C, X[n])
                S = Cn
                while True:
                    print("A calcular majorante...")
                    #passo 5.2
                    A = And(S, trans(X[n], Y[m]))
                    if solver.solve([A,B]):
                        break
                    else:
                        #passo 5.3
                        C = binary_interpolant(A,B)
                        #passo 5.4
                        Cn = rename(C, X[n])
                        if solver.solve(Cn, Not(S)):
                            #passo 5.5
                            S = Or(S, Cn)
                        else:
                            print("o sistema é seguro")
                            return
        return

model_checking(var, init, trans, error, 50, 50)